<a href="https://colab.research.google.com/github/thousandoaks/TopicsInDataAnalysis/blob/main/MisinformationChallenge/FakeNewsClassifier_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install datasets

     |████████████████████████████████| 311 kB 17.9 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 52.2 MB/s 
     |████████████████████████████████| 1.1 MB 62.8 MB/s 
     |████████████████████████████████| 133 kB 56.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 47.2 MB/s 
     |████████████████████████████████| 271 kB 10.8 MB/s 


In [4]:
import sys
!{sys.executable} -m pip install transformers

     |████████████████████████████████| 3.5 MB 16.6 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 6.8 MB 28.4 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [55]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

from transformers import  DataCollatorWithPadding

from transformers import  TrainingArguments, Trainer

import tensorflow as tf

## 1. Lets load a real dataset

Refer to:
https://huggingface.co/datasets/Annabelleabbott/real-fake-news-workshop/viewer/Annabelleabbott--real-fake-news-workshop/train

In [6]:
raw_datasets = load_dataset("GonzaloA/fake_news")

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Using custom data configuration GonzaloA--fake_news-5fb6f2534da0da6d


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/38.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/GonzaloA--fake_news-5fb6f2534da0da6d/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})

In [8]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'Unnamed: 0': 0,
 'label': 0,
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the caption underneath the Imgur upload reads,  Ted Cruz in drag on Maury. Here is an image from the offici

# 2. Let's preprocess the dataset

In [9]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, truncation=True,padding=True)
batch_size = 16

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [10]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [12]:
task = "sst2"

In [13]:
raw_datasets['train'][0]['title']

' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)'

In [14]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {raw_datasets['train'][0]['title']}")
else:
    print(f"Sentence 1: {raw_datasets['train'][0][sentence1_key]}")
    print(f"Sentence 2: {raw_datasets['train'][0][sentence2_key]}")

Sentence:  ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)


In [15]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples['title'], truncation=True,padding=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [16]:
encoded_dataset = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

# 3. Let's fine tune the model

In [17]:
num_labels = 2

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
training_args = TrainingArguments("test-trainer", num_train_epochs=1,max_steps=100)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [30]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    
   
)



max_steps is given, it will override any value given in num_train_epochs


### Please bear in mind that we are training in 1 epoch and 100 steps, this is too low for any realisting classifier

In [31]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, title, text.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24353
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.10409204483032226, metrics={'train_runtime': 310.3766, 'train_samples_per_second': 2.578, 'train_steps_per_second': 0.322, 'total_flos': 12685823224416.0, 'train_loss': 0.10409204483032226, 'epoch': 0.03})

# 4. Let's save the model

In [33]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained('./test-trainer/')

Configuration saved in ./test-trainer/config.json
Model weights saved in ./test-trainer/pytorch_model.bin


# 5. Let's fine tune the model and have it benchmarked

In [40]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [41]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
   
)


max_steps is given, it will override any value given in num_train_epochs


In [42]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, title, text.
***** Running Evaluation *****
  Num examples = 8117
  Batch size = 8


{'eval_accuracy': 0.9308857952445485,
 'eval_loss': 0.3604815602302551,
 'eval_runtime': 731.4796,
 'eval_samples_per_second': 11.097,
 'eval_steps_per_second': 1.388}

# 6. Let's do some prediction

In [45]:
raw_train_dataset[0]

{'Unnamed: 0': 0,
 'label': 0,
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the caption underneath the Imgur upload reads,  Ted Cruz in drag on Maury. Here is an image from the offici

In [62]:
raw_train_dataset[210]

{'Unnamed: 0': 210,
 'label': 1,
 'text': 'KABUL (Reuters) - The United Nations mission in Afghanistan said at least 10 civilians may have been killed by an air strike in the northern city of Kunduz last week, despite a U.S. military investigation that found no civilian deaths. In a series of messages on Twitter, the U.N. Assistance Mission in Afghanistan said interviews with multiple survivors, medics, elders and others gave it strong reason to believe that civilians were among the victims of Saturday s attack.  Credible reports that at least 10 civilians killed in Kunduz Afghanistan air strike 4 Nov, UNAMA initial findings show,  it said. The statement contradicts comments from some local and Afghan military officials as well as a U.S. statement on Tuesday which said that an investigation had found no evidence of civilian deaths. The episode underlines the lack of clarity over reports of civilian casualties, an issue which is likely to become increasingly prominent as the United Stat

In [63]:
NotFakeExample=raw_train_dataset[0]['title']
FakeExample=raw_train_dataset[210]['title']

In [57]:
inputs = tokenizer(NotFakeExample, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
outputs

SequenceClassifierOutput([('loss', tensor(9.2372, grad_fn=<NllLossBackward0>)),
                          ('logits',
                           tensor([[ 5.2836, -3.9534]], grad_fn=<AddmmBackward0>))])

In [64]:
inputs = tokenizer(FakeExample, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
outputs

SequenceClassifierOutput([('loss', tensor(0.0016, grad_fn=<NllLossBackward0>)),
                          ('logits',
                           tensor([[-3.0529,  3.4013]], grad_fn=<AddmmBackward0>))])